We now work directly with the song genre year dataset after data aggregation and cleaning in dataset.ipynb.
We have included the necessary files for running this notebook so that it can be run independently of data handling and doesn't need to be downloaded. (site might go down etc.)

Before you run the notebook, create a virtual environment as mentioned in README.txt and make sure you are using that environment when running the cells.
Install requirements.txt if not already done by running following cell.

In [3]:
!pip3 install -r requirements.txt

In [4]:
import pandas as pd

pd.options.mode.chained_assignment = None
msd_df = pd.read_csv("song_genre_year.csv")

In [5]:
df = msd_df[["song_id"]]

In [6]:
msd_df.isna().sum()

Unnamed: 0.1                           0
track_id                               0
genre                                  0
Unnamed: 0                             0
analyzer_version                  152793
artist_7digitalid                      0
artist_familiarity                     0
artist_hotttnesss                      0
artist_id                              0
artist_latitude                    94930
artist_location                    55595
artist_longitude                   94930
artist_mbid                          719
artist_name                            0
artist_playmeid                        0
idx_artist_terms                       0
idx_similar_artists                    0
release                                2
release_7digitalid                     0
song_hotttnesss                    39716
song_id                                0
title                                  0
track_7digitalid                       0
analysis_sample_rate                   0
audio_md5       

In [7]:
msd_df.eq(0).sum(axis=0)

Unnamed: 0.1                           1
track_id                               0
genre                                  0
Unnamed: 0                             0
analyzer_version                       0
artist_7digitalid                      0
artist_familiarity                   108
artist_hotttnesss                   1214
artist_id                              0
artist_latitude                        0
artist_location                        0
artist_longitude                       0
artist_mbid                            0
artist_name                            0
artist_playmeid                        0
idx_artist_terms                  152793
idx_similar_artists               152793
release                                0
release_7digitalid                     0
song_hotttnesss                     3957
song_id                                0
title                                  0
track_7digitalid                       0
analysis_sample_rate                   0
audio_md5       

Due to lot of missing values or zeroes in a lot of features we narrow down to the following columns to be used for content based filtering further.

In [8]:
msd_df[["song_id", "genre", "year", "key", "mode", "tempo", "loudness"]].describe()

,year,key,mode,tempo,loudness
count,152793.000000,152793.000000,152793.000000,152793.000000,152793.000000
mean,1997.938721,5.324851,0.663231,125.436801,-9.291391
std,11.204019,3.588301,0.472607,34.157529,4.626484
min,1922.000000,0.000000,0.000000,0.000000,-55.751000
25%,1994.000000,2.000000,0.000000,99.423000,-11.658000
50%,2002.000000,5.000000,1.000000,122.987000,-8.297000
75%,2006.000000,9.000000,1.000000,146.523000,-5.930000
max,2010.000000,11.000000,1.000000,269.510000,4.150000


We will be dealing with a subset of the user song interactions dataset. It consists of user_id, song_id, listen_count triplets i.e. what song a user listened to and how many times. 

There are 48 Million user song interactions! Training on the whole is out of the scope of this project. Since the crux of our algorithm relies on creating a user-item interaction matrix it takes up huge amounts of RAM.

To just train 500K user song interactions we required 128 GB RAM on 1 CPU node on a PACE cluster provided by Georgia Tech.

The user_play_counts.csv is a subset of the train_triplets.txt which we downloaded in dataset.ipynb and contains 500k rows out of 48M.

To successfully run the demo on your system or colab, we recommend keeping nrows=10_000 to 100_000.

In [9]:
user_play_counts_df = pd.read_csv("user_play_counts.csv", index_col=0, nrows=10_000)
user_play_counts_df

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
...,...,...,...
9995,8caf9a87e266a22298bd977a63489d008af241c5,SOETKSY12A8C13C666,12
9996,8caf9a87e266a22298bd977a63489d008af241c5,SOFPXJZ12A6D4F6444,1
9997,8caf9a87e266a22298bd977a63489d008af241c5,SOFRCGW12A81C21EA6,3
9998,8caf9a87e266a22298bd977a63489d008af241c5,SOFUVPZ12A6D4FCEA3,2


Since we are working on a subset of the song metadata after filtering out songs, we need to make sure the user song interactions have those song_ids available. Hence we perform an inner join.

In [10]:
df = user_play_counts_df.merge(df, on="song_id")

Following algorithm is a modified version for the million song dataset from the open source code available for the research paper: https://arxiv.org/abs/1905.03375. This algorithm applies only collaborative filtering as explained in the report. We will be training and evaluating this model and treating it as the baseline model to compare performance against.

In [11]:
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from multiprocessing import Pool, cpu_count


class EASE:
    def __init__(self):
        self.user_enc = LabelEncoder()
        self.item_enc = LabelEncoder()

    def _get_users_and_items(self, df):
        users = self.user_enc.fit_transform(df.loc[:, "user_id"])
        items = self.item_enc.fit_transform(df.loc[:, "song_id"])
        return users, items

    def fit(self, df, lambda_: float = 0.5, implicit=True):
        """
        df: pandas.DataFrame with columns user_id, song_id, and (listen_count)
        lambda_: l2-regularization term
        implicit: if True, listen_count is ignored and taken as 1, else normalized listen_count is used
        """
        users, items = self._get_users_and_items(df)
        values = (
            np.ones(df.shape[0])
            if implicit
            else df["listen_count"].to_numpy() / df["listen_count"].max()
        )

        X = csr_matrix((values, (users, items)))
        self.X = X

        G = X.T.dot(X).toarray()
        diagIndices = np.diag_indices(G.shape[0])
        G[diagIndices] += lambda_
        P = np.linalg.inv(G)
        B = P / (-np.diag(P))
        B[diagIndices] = 0

        self.B = B
        self.pred = X.dot(B)

    def predict(self, train, users, items, k):
        items = self.item_enc.transform(items)
        dd = train.loc[train.user_id.isin(users)]
        dd["ci"] = self.item_enc.transform(dd.song_id)
        dd["cu"] = self.user_enc.transform(dd.user_id)
        g = dd.groupby("cu")
        with Pool(cpu_count()) as p:
            user_preds = p.starmap(
                self.predict_for_user,
                [(user, group, self.pred[user, :], items, k) for user, group in g],
            )
        df = pd.concat(user_preds)
        df["song_id"] = self.item_enc.inverse_transform(df["song_id"])
        df["user_id"] = self.user_enc.inverse_transform(df["user_id"])
        return df

    @staticmethod
    def predict_for_user(user, group, pred, items, k):
        watched = set(group["ci"])
        candidates = [item for item in items if item not in watched]
        pred = np.take(pred, candidates)

        res = np.argpartition(pred, -k)[-k:]
        r = pd.DataFrame(
            {
                "user_id": [user] * len(res),
                "song_id": np.take(candidates, res),
                "score": np.take(pred, res),
            }
        ).sort_values("score", ascending=False)
        return r

Now, following model is the novel hybrid filtering approach we introduce to EASE and call it EASEwithFeatures. While training the model (fit) we separately calculate cosine similarity for the song_features. While predicting the score for recommendations for a user we use the combined score of collaborative filtering and weighted score of the features of the song and this is how we incorporate a hybrid filtering recommendation system.

In [12]:
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from multiprocessing import Pool, cpu_count


class EASEwithFeatures:
    def __init__(self):
        self.user_enc = LabelEncoder()
        self.item_enc = LabelEncoder()
        self.song_features = None

    def _get_users_and_items(self, df):
        users = self.user_enc.fit_transform(df.loc[:, "user_id"])
        items = self.item_enc.fit_transform(df.loc[:, "song_id"])
        return users, items

    def fit(self, df, lambda_: float = 0.5, implicit=True, song_features=None):
        """
        df: pandas.DataFrame with columns user_id, song_id, and (listen_count)
        lambda_: l2-regularization term
        implicit: if True, listen_count is ignored and taken as 1, else normalized listen_count is used
        """
        users, items = self._get_users_and_items(df)
        values = (
            np.ones(df.shape[0])
            if implicit
            else df["listen_count"].to_numpy() / df["listen_count"].max()
        )

        X = csr_matrix((values, (users, items)))
        self.X = X
        self.song_features = csr_matrix(song_features.drop("song_id", axis=1))

        G = X.T.dot(X).toarray()
        diagIndices = np.diag_indices(G.shape[0])
        G[diagIndices] += lambda_
        P = np.linalg.inv(G)
        B = P / (-np.diag(P))
        B[diagIndices] = 0

        self.B = B
        self.pred = X.dot(B)

    def predict(self, train, users, items, k):
        items = self.item_enc.transform(items)
        dd = train.loc[train.user_id.isin(users)]
        dd.loc[:, "ci"] = self.item_enc.transform(dd["song_id"])
        dd.loc[:, "cu"] = self.user_enc.transform(dd["user_id"])
        g = dd.groupby("cu")
        with Pool(cpu_count()) as p:
            user_preds = p.starmap(
                self.predict_for_user,
                [
                    (user, group, self.pred[user, :], items, k, self.song_features)
                    for user, group in g
                ],
            )
        df = pd.concat(user_preds)
        df["song_id"] = self.item_enc.inverse_transform(df["song_id"])
        df["user_id"] = self.user_enc.inverse_transform(df["user_id"])
        return df

    @staticmethod
    def predict_for_user(user, group, pred, items, k, song_features):
        watched = set(group["ci"])
        candidates = [item for item in items if item not in watched]
        pred = np.take(pred, candidates)

        song_features_indices = np.array(candidates)
        song_features_values = song_features[song_features_indices, :]
        combined_scores = pred

        num_features = song_features_values.shape[1]
        for i in range(num_features):
            feature_scores = song_features_values[:, i] * 0.001
            combined_scores += feature_scores.toarray().ravel()

        res = np.argpartition(combined_scores, -k)[-k:]
        r = pd.DataFrame(
            {
                "user_id": [user] * len(res),
                "song_id": np.take(candidates, res),
                "score": combined_scores[res],
            }
        ).sort_values("score", ascending=False)
        return r

Let's select the features from song metadata and apply necessary transformations before feeding into the model.

In [13]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

song_features = msd_df[
    ["song_id", "genre", "song_hotttnesss", "year", "key", "mode", "tempo", "loudness"]
]
mean_hotttnesss = song_features["song_hotttnesss"].mean()
song_features.loc[:, "song_hotttnesss"].fillna(mean_hotttnesss, inplace=True)

# numerically encode genre
label_encoder = LabelEncoder()
song_features["genre"] = label_encoder.fit_transform(song_features["genre"])

# normalize numerical columns
scaler = MinMaxScaler()
song_features[["song_hotttnesss"]] = scaler.fit_transform(
    song_features[["song_hotttnesss"]]
)
song_features[["genre"]] = scaler.fit_transform(song_features[["genre"]])
song_features[["year"]] = scaler.fit_transform(song_features[["year"]])
song_features[["key"]] = scaler.fit_transform(song_features[["key"]])
song_features[["tempo"]] = scaler.fit_transform(song_features[["tempo"]])
song_features[["loudness"]] = scaler.fit_transform(song_features[["loudness"]])

song_features

<bound method NDFrame.head of                    song_id     genre  song_hotttnesss      year       key  \
0       SOBLFFE12AF72AA5BA  0.928571         0.733372  0.988636  0.090909   
1       SOCSNVI12A8C13ECC2  0.928571         0.375984  0.545455  0.636364   
2       SONHOTT12A8C13493C  0.928571         0.526079  0.681818  0.000000   
3       SOIGICF12A8C141BC5  0.142857         0.468998  0.931818  1.000000   
4       SOEPPTV12AC4689D86  0.571429         0.526079  0.943182  0.454545   
...                    ...       ...              ...       ...       ...   
152788  SOFBODT12AB0180C40  0.928571         0.253835  0.840909  0.909091   
152789  SOISCAF12A8C142C8C  0.928571         0.360371  0.943182  0.909091   
152790  SOULKJA12A8C140620  0.928571         0.686652  0.920455  0.181818   
152791  SOSULQJ12A8C144B79  0.214286         0.530026  0.556818  0.636364   
152792  SOZPUEF12AF72A9F2A  0.142857         0.392009  0.897727  0.090909   

        mode     tempo  loudness  
0         

Let's now evaluate our models.
We will be calculating hit rate, arhr (average reciprocal hit rate), novelty, precision and recall and comparing EASE and EASEwithFeatures. All of these have been explained in brief in final report.

In [14]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
all_song_ids = train_df["song_id"].unique()


def evaluate_model(ease_model):
    hits = 0
    total_users = 0
    arhr_sum = 0
    novelty_sum = 0
    novelty_count = 0
    total_correct_recommended = 0
    total_recommended = 0
    total_relevant = 0
    k = 10

    grouped_test_df = test_df.groupby("user_id")

    for user_id, user_data in tqdm(grouped_test_df, desc="Evaluating Model"):
        actual_interactions = set(user_data["song_id"])

        try:
            recommendations = ease_model.predict(train_df, [user_id], all_song_ids, k)
            recommended_songs = set(recommendations["song_id"])

            hits += len(actual_interactions & recommended_songs) > 0

            intersection = list(actual_interactions & recommended_songs)
            if intersection:
                intersection_rank = recommendations.index[
                    recommendations["song_id"] == intersection[0]
                ][0]
                reciprocal_rank = 1 / (intersection_rank + 1)
                arhr_sum += reciprocal_rank

            song_user_counts = train_df[train_df["song_id"].isin(recommended_songs)][
                "user_id"
            ].nunique()
            novelty_sum += song_user_counts
            novelty_count += len(recommended_songs)

            total_recommended += len(recommended_songs)
            total_relevant += len(actual_interactions)
            total_correct_recommended += len(actual_interactions & recommended_songs)

            total_users += 1
        except Exception as e:
            pass

    hit_rate = hits / total_users if total_users > 0 else 0.0
    arhr = arhr_sum / total_users if total_users > 0 else 0
    novelty = novelty_sum / novelty_count if novelty_count > 0 else 0
    precision = (
        total_correct_recommended / total_recommended if total_recommended > 0 else 0.0
    )
    recall = total_correct_recommended / total_relevant if total_relevant > 0 else 0.0

    print("Hit Rate:", hit_rate)
    print("ARHR:", arhr)
    print("Novelty:", novelty)
    print("Precision:", precision)
    print("Recall:", recall)


ease_model = EASE()
print("Fitting EASE Model")
ease_model.fit(train_df, lambda_=0.5, implicit=True)
print("Evalution score for EASE")
evaluate_model(ease_model)

ease_model_with_features = EASEwithFeatures()
print("Fitting EASE with Features Model")
ease_model_with_features.fit(
    train_df, lambda_=0.5, implicit=True, song_features=song_features
)
print("Evalution score for EASE with features")
evaluate_model(ease_model_with_features)

Fitting EASE Model
Evalution score for EASE


Evaluating Model:  36%|███▋      | 53/146 [00:16<00:25,  3.65it/s]

Now we will be training on the whole dataset and generate recommendations for all users for the interactive UI.

In [ ]:
model = EASEwithFeatures()
model.fit(df, lambda_=0.5, implicit=True, song_features=song_features)


def generate_recommendations_for_all_users(model, users, all_items, k):
    all_recommendations = []
    for user in tqdm(users):
        recommendations = model.predict(df, [user], all_items, k)
        recommendations["rank"] = recommendations.groupby("user_id").cumcount() + 1
        all_recommendations.append(recommendations)

    all_recommendations = pd.concat(all_recommendations)
    return all_recommendations


all_song_ids = df["song_id"].unique()
all_users = df["user_id"].unique()

all_user_recommendations = generate_recommendations_for_all_users(
    model, all_users, all_song_ids, k=10
)
all_user_recommendations[["user_id", "song_id", "rank"]]

100%|██████████| 171/171 [00:58<00:00,  2.94it/s]


,user_id,song_id,rank
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAOAHZ12A8C13AAF1,1
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SORKFWO12A8C138D83,2
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHSOEO12A8C1314F9,3
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOUKJBT12A6701C4D6,4
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOGKBUP12A8AE46251,5
...,...,...,...
4,b4adf5ac8bcba59011df7be840dda1134a47cc30,SOHWTCP12A3F1EA9EB,6
1,b4adf5ac8bcba59011df7be840dda1134a47cc30,SOMXBSN12A6310E0DC,7
3,b4adf5ac8bcba59011df7be840dda1134a47cc30,SORWGRQ12A8C13FFA1,8
2,b4adf5ac8bcba59011df7be840dda1134a47cc30,SODZXQS12A6D4F6C0E,9


For the force directed graph on UI we also need to show song similarities after recommending top k recommendations. Hence we use normal cosine_similarity (content based filtering only) to satisfy this feature. 

Note: Doing this only for 100 songs for demo purposes. If you run for whole dataset there will be 1.5 million rows generated (took around an hour on PACE with 1 Node 8 cores 24 GB RAM which we have employed in our UI).

In [ ]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

song_ids = msd_df["song_id"][:100]


def find_similar_songs(song_id, k):
    song_index = song_features.index[song_features["song_id"] == song_id].tolist()[0]

    similarities = cosine_similarity(
        song_features.drop("song_id", axis=1).iloc[[song_index]],
        song_features.drop("song_id", axis=1),
    )

    similar_indices = similarities.argsort(axis=1)[:, : -k - 1 : -1]
    similar_song_ids = [
        song_features.iloc[i]["song_id"]
        for i in similar_indices[0]
        if song_features.iloc[i]["song_id"] != song_id
    ]

    similar_songs = [
        (sim_song_id, rank + 1) for rank, sim_song_id in enumerate(similar_song_ids[:k])
    ]
    return similar_songs


similar_songs_list = []
for song_id in tqdm(song_ids, desc="Recommend songs using song similarity"):
    similar_songs = find_similar_songs(song_id, k=10)
    for sim_song, rank in similar_songs:
        similar_songs_list.append(
            {"source_song_id": song_id, "target_song_id": sim_song, "rank": rank}
        )

similar_songs_df = pd.DataFrame(similar_songs_list)
similar_songs_df

Recommend songs using song similarity:   0%|          | 0/100 [00:00<?, ?it/s]

Recommend songs using song similarity: 100%|██████████| 100/100 [00:08<00:00, 11.82it/s]


,source_song_id,target_song_id,rank
0,SOBLFFE12AF72AA5BA,SOWFJJN12A6D4FC7B6,1
1,SOBLFFE12AF72AA5BA,SOUHCLN12A8AE46CD9,2
2,SOBLFFE12AF72AA5BA,SOSVMUX12AB0181BC9,3
3,SOBLFFE12AF72AA5BA,SOKGYUR12A6D4F7D55,4
4,SOBLFFE12AF72AA5BA,SOZWDIG12A8C139830,5
...,...,...,...
895,SOQAHLN12A8C13F853,SOTSRUQ12A81C22359,5
896,SOQAHLN12A8C13F853,SOKMUSD12A6D4F528D,6
897,SOQAHLN12A8C13F853,SOAAMAA12A58A7F88A,7
898,SOQAHLN12A8C13F853,SONPTBS12AB018E4B9,8
